In [2]:
import pandas as pd
import svgwrite

In [5]:
ideogram_df = pd.read_csv("hg38.ideogram.txt", sep="\t")

In [86]:
gie_color = {}
gie_color["gpos25"] = "#C8C8C8"
gie_color["gpos25_high"] = "#DCDCDC"
gie_color["gpos50"] = "#C8C8C8"
gie_color["gpos50_high"] = "#DCDCDC"
gie_color["gpos75"] = "#828282"
gie_color["gpos75_high"] = "#C6C6C6"
gie_color["gpos100"] = "#000000"
gie_color["gpos100_high"] = "#444444"
gie_color["gneg"] = "#FFFFFF"
gie_color["gneg_high"] = "#FFFFFF"
gie_color["acen"] = "#D92F27"
gie_color["acen_high"] = "#F7574F"
gie_color["gvar"] = "#AAAAFF"
gie_color["gvar_high"] = "#CCCCFF"
gie_color["stalk"] = "#647FA4"
gie_color["stalk_high"] = "#647FA4"

In [89]:
scaling = 100000

In [39]:
chromosomes = [f"chr{x}" for x in range(1, 23)] + ["chrX", "chrY"]

In [88]:
for chrom in chromosomes:
    
    chrom_df = ideogram_df[ideogram_df["#chrom"] == chrom].reset_index()
    
    dwg = svgwrite.Drawing(f"{chrom}.svg", size=(chrom_df["chromEnd"].max() / scaling, 100))

    ### make rounded rectangle as borders and use those for clipping
    # P
    rounded_rect_p = dwg.rect(
        insert=(0 / scaling, 0), 
        size=(chrom_df[chrom_df["gieStain"] == "acen"]["chromEnd"].min() / scaling, 100), 
        fill = "none",
        stroke="black", 
        stroke_width=2,
        rx = 30, ry = 30
    )
    # Q
    rounded_rect_q = dwg.rect(
        insert=(chrom_df[chrom_df["gieStain"] == "acen"]["chromStart"].max() / scaling, 0), 
        size=((chrom_df["chromEnd"].max() - chrom_df[chrom_df["gieStain"] == "acen"]["chromStart"].max()) / scaling, 100), 
        fill = "none",
        stroke="black", 
        stroke_width=2,
        rx = 30, ry = 30
    )  

    for gie_stain in gie_color:
        if not gie_stain.endswith("_high"):
            # make pretty gradient
            grad = dwg.linearGradient(start=("0%", "0%"), end=("0%", "100%"), id=f"grad_{gie_stain}")
            # Add color stops: top, middle, bottom
            grad.add_stop_color(0.0, gie_color[gie_stain])   # Top
            grad.add_stop_color(0.4, gie_color[f"{gie_stain}_high"])  # Middle
            grad.add_stop_color(0.5, gie_color[f"{gie_stain}_high"])  # Middle
            grad.add_stop_color(0.6, gie_color[f"{gie_stain}_high"])  # Middle
            grad.add_stop_color(1.0, gie_color[gie_stain])   # Bottom

            dwg.defs.add(grad)
        
    
    for idx, row in chrom_df.iterrows():

        # make rectangle for one stained region
        stained_region = dwg.rect(
            insert=(row["chromStart"] / scaling, 0), 
            size=((row["chromEnd"] - row["chromStart"]) / scaling, 100), 
            #fill=gie_color[row["gieStain"]],
            fill=f"url(#grad_{row['gieStain']})",
            id=row["name"]
        )

        # Define a clipPath
        clip = dwg.defs.add(dwg.clipPath(id=f"rounded_clip_{row['name']}"))
        if row["name"].startswith("p"):
            clip.add(rounded_rect_p)
        else:
            clip.add(rounded_rect_q)

        # add clip to stained region
        stained_region['clip-path'] = f"url(#rounded_clip_{row['name']})"

        dwg.add(stained_region)

    ### add rounded rectangle as border
    dwg.add(rounded_rect_p)
    dwg.add(rounded_rect_q)
    
    dwg.save()